In [1]:
!pip install jsonlines
!pip install transformers
!pip install datasets
!pip install --quiet https://github.com/PyTorchLightning/pytorch-lightning/releases/download/1.2.6/pytorch-lightning-1.2.6.tar.gz 
!pip install torchmetrics

     |████████████████████████████████| 2.5MB 7.1MB/s 
     |████████████████████████████████| 901kB 46.5MB/s 
     |████████████████████████████████| 3.3MB 53.9MB/s 
     |████████████████████████████████| 245kB 7.4MB/s 
     |████████████████████████████████| 245kB 49.0MB/s 
     |████████████████████████████████| 122kB 31.1MB/s 
     |████████████████████████████████| 2.0MB 8.4MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 829kB 9.4MB/s 
     |████████████████████████████████| 235kB 19.7MB/s 
     |████████████████████████████████| 276kB 27.5MB/s 
     |████████████████████████████████| 1.3MB 34.8MB/s 
     |████████████████████████████████| 143kB 53.0MB/s 
     |████████████████████████████████| 296kB 51.4MB/s 


In [2]:
import jsonlines
import json
import torch
from transformers import AutoModel, AutoTokenizer, Trainer, TrainingArguments, T5ForConditionalGeneration, get_linear_schedule_with_warmup, AdamW, T5Tokenizer
import pandas as pd
from datasets import Dataset, DatasetDict, load_metric
import pytorch_lightning as pl
# from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint
import torch.nn.functional as F
from torchmetrics import Precision, Recall, Accuracy

In [3]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('drive/MyDrive/NeuralDB/data/v2.4_25/train.jsonl', 'r') as json_file:
    json_list = list(json_file)

train_dbs, test_dbs, dev_dbs = [], [], []

for json_str in json_list:
    result = json.loads(json_str)
    train_dbs.append(result)
    # print(f"result: {result}")
    # print(isinstance(result, dict))

with open('drive/MyDrive/NeuralDB/data/v2.4_25/test.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    test_dbs.append(result)


with open('drive/MyDrive/NeuralDB/data/v2.4_25/dev.jsonl', 'r') as json_file:
    json_list = list(json_file)

for json_str in json_list:
    result = json.loads(json_str)
    dev_dbs.append(result)

In [5]:
#@title Metrics Definition

train_acc = Accuracy()
test_acc = Accuracy()
val_acc = Accuracy()

train_prec = Precision()
test_prec = Precision()
val_prec = Precision()

train_rec = Recall()
test_rec = Recall()
val_rec = Recall()

In [6]:
MODEL_NAME = "t5-small"

# We need to create the model and tokenizer
# model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [7]:
def get_facts_perfectIR(db, query):
    triples = []

    facts = query['facts']

    for fact in facts:
        fact_info = db['metadata']['raw'][fact[0]]

        # for every triplet of a db fact
        for parse in fact_info['parses']:
            triple = []
            triple_codes = []
            # get triple's text and codes of the triple's terms
            for term in parse:
                triple.append(term[0])
                triple_codes.append(term[1][0])

            # if the triple's code combination exists in valid_hypotheses add triple to query_triples
            if triple_codes in fact_info['valid_hypotheses']:
                triples.append(' '.join(triple))
        

    return ' '.join(triples)


In [8]:
train_dataset = []

for db in train_dbs:
    for query in db['queries']:
      if query['type'] == 'bool' or (query['type'] == 'set' and len(query['answer']) == 1):
        query_text = query['query']
        if query_text != '':
          query_facts = get_facts_perfectIR(db, query)
          answer = ' '.join([str(a) for a in query['answer']])

          train_dataset.append([query_text, query_facts, answer])

In [9]:
test_dataset = []
types = set()

for db in test_dbs:
    for query in db['queries']:
      types.add(query['type'])
      if query['type'] == 'bool' or (query['type'] == 'set' and len(query['answer']) == 1):
        query_text = query['query']
        if query_text != '':
          query_facts = get_facts_perfectIR(db, query)
          answer = ' '.join([str(a) for a in query['answer']])

          test_dataset.append([query_text, query_facts, answer])

In [ ]:
types

{'argmax', 'argmin', 'bool', 'count', 'max', 'min', 'set'}

In [10]:
dev_dataset = []
for db in dev_dbs:
    for query in db['queries']:
      if query['type'] == 'bool' or (query['type'] == 'set' and len(query['answer']) == 1):
        query_text = query['query']
        if query_text != '':
          query_facts = get_facts_perfectIR(db, query)
          answer = ' '.join([str(a) for a in query['answer']])

          dev_dataset.append([query_text, query_facts, answer])

In [11]:
len(train_dataset)

7240

In [12]:
train_df = pd.DataFrame(train_dataset, columns = ['query', 'facts', 'answers'])
test_df = pd.DataFrame(test_dataset, columns = ['query', 'facts', 'answers'])
dev_df = pd.DataFrame(dev_dataset, columns = ['query', 'facts', 'answers'])

# train_d = Dataset.from_pandas(train_df)
# test_d = Dataset.from_pandas(test_df)
# dev_d = Dataset.from_pandas(dev_df)

# dataset_dict = {"train": train_d,
#            "test": test_d,
#            "unsupervised": dev_d}

# d = DatasetDict(dataset_dict)
# d

In [ ]:
def preprocess_function(examples):
    model_inputs = tokenizer(examples["query"], examples["facts"], padding="max_length", truncation=True, max_length=396, return_attention_mask=True, add_special_tokens=True, return_tensors="pt" )

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["answers"], max_length=32, padding='max_length', truncation=True, return_attention_mask=True, add_special_tokens=True, return_tensors="pt")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = d.map(preprocess_function, batched=True)

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['answers', 'attention_mask', 'facts', 'input_ids', 'labels', 'query'],
        num_rows: 5188
    })
    test: Dataset({
        features: ['answers', 'attention_mask', 'facts', 'input_ids', 'labels', 'query'],
        num_rows: 761
    })
    unsupervised: Dataset({
        features: ['answers', 'attention_mask', 'facts', 'input_ids', 'labels', 'query'],
        num_rows: 917
    })
})

In [13]:
from torch.utils.data import Dataset, DataLoader

class NeuralDBDataset(Dataset):
  def __init__(self, data:pd.DataFrame, tokenizer:T5Tokenizer, source_max_token_len: int = 396, target_max_token_len: int = 32):
    
    self.data =  data
    self.tokenizer =  tokenizer
    self.source_max_token_len =  source_max_token_len
    self.target_max_token_len =  target_max_token_len

  
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]

    source_encoding = tokenizer(data_row['query'], 
                                data_row['facts'], 
                                max_length=self.source_max_token_len, 
                                padding='max_length', 
                                truncation="only_second", 
                                return_attention_mask=True, 
                                add_special_tokens=True,
                                return_tensors="pt")
    
    target_encoding = tokenizer(data_row['answers'],
                                max_length=self.target_max_token_len,
                                padding='max_length',
                                truncation=True,
                                return_attention_mask=True,
                                add_special_tokens=True,
                                return_tensors="pt")

    labels = target_encoding['input_ids']


    return dict(
        question=data_row['query'],
        facts=data_row['facts'],
        answer=data_row['answers'],
        input_ids=source_encoding["input_ids"].flatten(),
        attention_mask=source_encoding['attention_mask'].flatten(),
        labels=labels.flatten()
    )

In [14]:
train_dataset = NeuralDBDataset(train_df, tokenizer)

for data in train_dataset:
  print("Question: ", data['question'])
  print("Answer text: ", data['answer'])
  print("Input_ids: ", data['input_ids'][:10])
  print("Labels: ", data['labels'][:10])
  break

Question:  Did Iain Banks write The White Masai?
Answer text:  FALSE
Input_ids:  tensor([3963,   27,    9,   77, 1925,    7, 1431,   37, 1945, 6664])
Labels:  tensor([  377, 20525,   427,     1,     0,     0,     0,     0,     0,     0])


In [15]:
class NeuralDBDataModule(pl.LightningDataModule):
  def __init__(self, train_df: pd.DataFrame, test_df: pd.DataFrame, dev_df: pd.DataFrame, tokenizer:T5Tokenizer, batch_size: int = 8, source_max_token_len: int = 396, target_max_token_len: int = 32):
    super().__init__()
    self.train_df = train_df
    self.test_df = test_df
    self.dev_df = dev_df
    self.tokenizer = tokenizer
    self.batch_size = batch_size
    self.source_max_token_len = source_max_token_len
    self.target_max_token_len = target_max_token_len

  def setup(self):
    self.train_dataset = NeuralDBDataset(self.train_df,
                                        self.tokenizer,
                                        self.source_max_token_len,
                                        self.target_max_token_len)

    self.test_dataset = NeuralDBDataset(self.test_df,
                                        self.tokenizer,
                                        self.source_max_token_len,
                                        self.target_max_token_len)
    
    self.dev_dataset = NeuralDBDataset(self.dev_df,
                                        self.tokenizer,
                                        self.source_max_token_len,
                                        self.target_max_token_len)
 
  def train_dataloader(self):
    return DataLoader(
        self.train_dataset,
        batch_size=self.batch_size,
        shuffle=True,
        num_workers=4
        )
  def val_dataloader(self):
    return DataLoader(
        self.dev_dataset,
        batch_size=self.batch_size,
        num_workers=4
        )

  def test_dataloader(self):
    return DataLoader(
        self.test_dataset,
        batch_size=1,
        num_workers=4
        )

In [21]:
BATCH_SIZE = 8
N_EPOCHS = 3

data_module = NeuralDBDataModule(train_df, test_df, dev_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [17]:
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict = True)

In [18]:
class NeuralDBModel(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict=True)

    self.train_acc = Accuracy()
    self.test_acc = Accuracy()
    self.val_acc = Accuracy()

    self.train_prec = Precision()
    self.test_prec = Precision()
    self.val_prec = Precision()

    self.train_rec = Recall()
    self.test_rec = Recall()
    self.val_rec = Recall()


  def forward(self, input_ids, attention_mask, labels=None):
    output = self.model(
        input_ids, 
        attention_mask=attention_mask,
        labels=labels)

    return output.loss, output.logits

  def training_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)

    # generated_ids = self.model.generate(
    #     input_ids=input_ids,
    #     attention_mask=attention_mask,
    #     num_beams=1,  # greedy search
    #     max_length=80,
    #     repetition_penalty=2.5,
    #     early_stopping=True,
    #     use_cache=True)
    
    # preds = [
    #         tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    #         for generated_id in generated_ids
    # ]

    # self.train_acc(preds, labels)
    self.log("train_loss", loss, prog_bar=True, logger=True, on_step=True, on_epoch=True)
    return {"loss": loss, "predictions":outputs, "labels": labels}

  
  # def training_epoch_end(self, outs):
  #   self.log('train_acc', self.model.train_acc.compute())

  def validation_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)

    # generated_ids = self.model.generate(
    #     input_ids=input_ids,
    #     attention_mask=attention_mask,
    #     num_beams=1,  # greedy search
    #     max_length=80,
    #     repetition_penalty=2.5,
    #     early_stopping=True,
    #     use_cache=True)
    
    # preds = [
    #         tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    #         for generated_id in generated_ids
    # ]

    # self.val_acc(preds, labels)
    self.log("val_loss", loss, prog_bar=True, logger=True, on_step=True, on_epoch=True)
    result = {'val_loss': loss}
    return result

  # def evaluation_epoch_end(self, outs):
  #   self.log('train_acc', self.val_acc.compute())

  def test_step(self, batch, batch_idx):
    input_ids = batch['input_ids']
    attention_mask=batch['attention_mask']
    labels = batch['labels']
    loss, outputs = self(input_ids, attention_mask, labels)

    # generated_ids = self.generate(
    #     input_ids=input_ids,
    #     attention_mask=attention_mask,
    #     num_beams=1,  # greedy search
    #     max_length=80,
    #     repetition_penalty=2.5,
    #     early_stopping=True,
    #     use_cache=True)
    
    # preds = [
    #         tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    #         for generated_id in generated_ids
    # ]

    # self.test_acc(preds, labels)
    self.log("test_loss", loss, prog_bar=True, logger=True, on_step=True, on_epoch=True)
    result = {'test_loss': loss}
    return result

  # def test_epoch_end(self, outs):
  #   self.log('train_acc', self.test_acc.compute())

  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=0.0001)
    return optimizer

In [19]:
model = NeuralDBModel()

In [20]:
import os
checkpoint_callback = ModelCheckpoint(
    dirpath=os.getcwd()+"/drive/MyDrive/NeuralDB/checkpoints/",
    filename="best-checkpoint",
    verbose=True,
    monitor="val_loss",
    mode="min"
)

In [23]:
trainer = pl.Trainer(
    #logger = logger,
    callbacks=checkpoint_callback,
    max_epochs=N_EPOCHS,
    gpus=1,
    progress_bar_refresh_rate = 30
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [24]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                       | Params
----------------------------------------------------------
0 | model      | T5ForConditionalGeneration | 60.5 M
1 | train_acc  | Accuracy                   | 0     
2 | test_acc   | Accuracy                   | 0     
3 | val_acc    | Accuracy                   | 0     
4 | train_prec | Precision                  | 0     
5 | test_prec  | Precision                  | 0     
6 | val_prec   | Precision                  | 0     
7 | train_rec  | Recall                     | 0     
8 | test_rec   | Recall                     | 0     
9 | val_rec    | Recall                     | 0     
----------------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker p

Epoch 0, global step 904: val_loss reached 0.02609 (best 0.02609), saving model to "/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint-v1.ckpt" as top 1


Epoch 1, step 1809: val_loss was not in top 1


Epoch 2, global step 2714: val_loss reached 0.01635 (best 0.01635), saving model to "/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint-v1.ckpt" as top 1


1

In [25]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 4.321052529121516e-06, 'test_loss_epoch': 0.02453053742647171}
--------------------------------------------------------------------------------


[{'test_loss': 4.321052529121516e-06, 'test_loss_epoch': 0.02453053742647171}]

In [27]:
trained_model = NeuralDBModel.load_from_checkpoint("/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint.ckpt")
trained_model.freeze() # 

In [28]:
def generate_answer(question):
  source_encoding=tokenizer(
      question["query"],
      question['facts'],
      max_length = 396,
      padding="max_length",
      truncation="only_second",
      return_attention_mask=True,
      add_special_tokens=True,
      return_tensors="pt"

  )

  generated_ids = trained_model.model.generate(
      input_ids=source_encoding["input_ids"],
      attention_mask=source_encoding["attention_mask"],
      num_beams=1,  # greedy search
      max_length=80,
      repetition_penalty=2.5,
      early_stopping=True,
      use_cache=True)
  
  preds = [
          tokenizer.decode(generated_id, skip_special_tokens=True, clean_up_tokenization_spaces=True)
          for generated_id in generated_ids
  ]

  return "".join(preds)

In [37]:
sample_question = dev_df.iloc[100]
sample_question["query"]

'Is Günther Happich male?'

In [39]:
generate_answer(sample_question)  # Predicted answer

''

In [40]:
answers = []
for i in range(len(dev_df)):
  answers.append(generate_answer(dev_df.iloc[i]) )

In [41]:
df = dev_df.copy()

In [42]:
df['preds'] = answers

In [ ]:
len(df)

In [43]:
len(df[df['answers'] == df['preds']])/len(df)

0.9100401606425703

In [47]:
df.head(40)

,query,facts,answers,preds
0,Was the alma mater of someone from British Ind...,Humayun Azad employer University of Dhaka Huma...,TRUE,TRUE
1,Did Sigmund Freud study at Conservatoire natio...,Sigmund Freud residence London Sigmund Freud e...,FALSE,FALSE
2,Is the population of Auggen +2477?,,TRUE,TRUE
3,Is Johnny Ballantyne a member of Scottish Foot...,Johnny Ballantyne member of sports team Scotti...,TRUE,TRUE
4,Did someone who is employed by University of V...,Sigmund Freud residence London Sigmund Freud e...,TRUE,TRUE
5,Did Charles L. Bestor get their degree from De...,Charles L. Bestor employer Juilliard School Ch...,FALSE,FALSE
6,Is Emil Tîmbur part of Stade Toulousain?,,,
7,Does Antoni Muntadas have the nationality of S...,Antoni Muntadas occupation university teacher ...,TRUE,TRUE
8,Who has a career as a film actor?,Lance Henriksen occupation Screenwriter Lance ...,Lance Henriksen,Lance Henriksen
9,Is Jazmín Zepeda Burgos a citizen of Mexico?,Jazmín Zepeda Burgos occupation Politician Jaz...,TRUE,TRUE


In [ ]:
!nvidia-smi

Fri Jul  2 15:42:00 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    60W / 149W |   1921MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name       | Type                       | Params
----------------------------------------------------------
0 | model      | T5ForConditionalGeneration | 60.5 M
1 | train_acc  | Accuracy                   | 0     
2 | test_acc   | Accuracy                   | 0     
3 | val_acc    | Accuracy                   | 0     
4 | train_prec | Precision                  | 0     
5 | test_prec  | Precision                  | 0     
6 | val_prec   | Precision                  | 0     
7 | train_rec  | Recall                     | 0     
8 | test_rec   | Recall                     | 0     
9 | val_rec    | Recall                     | 0     
----------------------------------------------------------
60.5 M    Trainable params
0         Non-trainable params
60.5 M    Total params
242.026   Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker p

Epoch 0, global step 648: val_loss reached 0.02497 (best 0.02497), saving model to "/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint-v4.ckpt" as top 1


Epoch 1, global step 1297: val_loss reached 0.02184 (best 0.02184), saving model to "/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint-v4.ckpt" as top 1


Epoch 2, global step 1946: val_loss reached 0.01760 (best 0.01760), saving model to "/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint-v4.ckpt" as top 1


Epoch 3, global step 2595: val_loss reached 0.01560 (best 0.01560), saving model to "/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint-v4.ckpt" as top 1


...

Epoch 6, global step 4542: val_loss reached 0.00003 (best 0.00003), saving model to "/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint-v4.ckpt" as top 1


Epoch 7, step 5191: val_loss was not in top 1


Epoch 8, step 5840: val_loss was not in top 1


Epoch 9, global step 6489: val_loss reached 0.00001 (best 0.00001), saving model to "/content/drive/MyDrive/NeuralDB/checkpoints/best-checkpoint-v4.ckpt" as top 1


1

In [ ]:
trainer.test()

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': 2.2351738238057806e-08, 'test_loss_epoch': 0.002627455862239003}
--------------------------------------------------------------------------------


[{'test_loss': 2.2351738238057806e-08,
  'test_loss_epoch': 0.002627455862239003}]